In [1]:
from query_deepseek import query_deepseek
from query_gpt import query_gpt
from schedule_parser import TimetableProcessor


In [3]:
processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False)
print(current_events)

事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 准备签证材料清单
日期: 2025-03-03
时间段: 20:00-21:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备会议材料
日期: 2025-03-04
时间段: 08:00-09:00
类型: 任务事项
截止日期：2025-03-04
重要程度：4

事项: 上课
日期: 2025-03-04
时间段: 11:00-17:00
类型: 固定日程
截止日期：2025-03-04
重要程度：3


In [4]:
user_prompt = input("你有什么想放进计划表的东西吗？")
#response = query_deepseek(user_prompt, current_events)
response = query_gpt(user_prompt, current_events)
print("\n模型回复：")
print(response)


【处理规则】

时间分配：
每日专注时段（默认 09:00-12:30;14:00-17:30）
单任务最长持续≤120分钟，间隔≥10分钟缓冲
冲突处理原则：截止时间近者优先，重要度相同则用时少者优先

【特殊规则】

自动预留截止时间前15%时长作为应急缓冲
连续工作>120分钟必须插入15分钟休息
每日保留19:00-21:00作为弹性时段（可调整）
输出时间精度保持±5分钟对齐

【输入示例】
当前时间：2024-03-14 16:00

当前时间表：
事项: 项目会议
日期: 2024-03-15
时间段: 09:00-10:30
类型: 固定日程
截止日期：2024-03-15
重要程度：5

事项: 完成报告
日期: 2024-03-16
时间段: 15:30-16:30
类型: 任务事项
截止日期：2024-03-18
重要程度：3

新增任务：
我后天下午三点要开个会，得准备个PPT


【输出示例】
新任务：开会并准备PPT
开会并准备PPT → 拆分为：
- 准备演讲稿（预计需两小时，时段：2024-03-15 15:00-17:00）
- 进行会议（预计需一小时，时段：2024-03-16 15:00-16:00）


日程建议：
事项: 完成报告
日期: 2024-03-17
时间段: 9:00-10:00
类型: 任务事项
截止日期：2024-03-18
重要程度：4
变动：更改

事项: 准备演讲稿
日期: 2024-03-15
时间段: 15:00-17:00
类型: 任务事项
截止日期：2024-03-16
重要程度：4
变动：新增

事项: 项目会议
日期: 2024-03-16
时间段: 15:00-16:00
类型: 固定日程
截止日期：2024-03-16
重要程度：5
变动：新增


请根据以上指令完成以下任务规划，并严格遵守输出格式，无需进行任何额外说明与解释：
【输入】
当前时间：2025-03-03 15:13

当前时间表：
事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度

In [5]:
# Process the response and update the database
summary = processor.process_events(response)
print(summary)

# This gets all the events in the database
formatted_output = processor.format_events_as_llm_output()
print(formatted_output)

# This gets all the events for a specific date
today_events = processor.get_events_for_date("2025-03-03")
today_formatted = processor.format_events_as_llm_output(today_events)
#print(today_formatted)


Extracted 9 events from LLM output
Event 1: 编写程序代码 - 2025-03-03 - 无
Event 2: 测试调试 - 2025-03-03 - 无
Event 3: 编写核心模块 - 2025-03-03 - 无
Event 4: 调试与测试 - 2025-03-03 - 无
Event 5: 准备签证材料清单 - 2025-03-03 - 无
Event 6: 准备会议材料 - 2025-03-04 - 无
Event 7: 上课 - 2025-03-04 - 无
Event 8: 选择花束 - 2025-03-05 - 新增
Event 9: 下单购买 - 2025-03-05 - 新增
{'added': 2, 'modified': 0, 'deleted': 0, 'unchanged': 7, 'skipped': 0, 'errors': [], 'warnings': []}
事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 准备签证材料清单
日期: 2025-03-03
时间段: 20:00-21:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备会议材料
日期: 2025-03-04
时间段: 08:00-09:00
类型: 任务事项
截止日期：2025-03-04
重要程度：4

事项: 上课
日期: 2025-03-04
时间段: 11:00-17:00
类型: 固定日程
截止日期：2025-03-04
重要程度：3

事项: 选择花束
日期: 2025-03-05
时间段: 19:00-19:30
类型: 任

In [4]:
# If there are duplicate events, remove them
result = processor.remove_duplicates()
print(f"Removed {result['removed_duplicates']} duplicate events")

Removed 4 duplicate events


In [8]:
formatted_output = processor.format_events_as_llm_output()
print(formatted_output)

事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5
